To automatically predict the congestion level on-board of public transport via open data eg. Trafiklab/Google and data pipeline powered by IBM and other open source community 

Step 1. Read the ID from station names

In [1]:
from collections import OrderedDict
import csv
import pandas as pd
import numpy as np
import requests 

dt = pd.read_excel('list_stop_sl_30.xlsx')
#dt = pd.DataFrame(dt)
r = [0 for i in range(len(dt.index))]



for index, row in dt.iterrows():
    urls = "https://api.resrobot.se/v2/location.name?key=YOURKEY&input=" + row['stopName']
    r[index] = requests.get(urls)

print(r[0].status_code)    


200


In [2]:
print(r[:2])
#status0 = np.array(r.status_code)
#print(np.unique(status0))


[<Response [200]>, <Response [200]>]


In [3]:
import xml.etree.ElementTree as ET
from lxml.etree import fromstring

dfcols = ['name','ID','lon','lat']
df = pd.DataFrame(columns = dfcols)
    
for el in r:
    root = ET.fromstring(el.content)

#for child in root.iter('*'):
#    print(child.attrib,child.attrib)

    for child in root.iter('{hafas_rest_v1}StopLocation'):
        name = child.attrib['name']
        ID = child.attrib['id']
        lon = child.attrib['lon']
        lat = child.attrib['lat']
        df = df.append(pd.Series([name, ID, lon,lat], index = dfcols), ignore_index = True)

print(df.shape)
df.head()

(124, 4)


,name,ID,lon,lat
0,Stavsnäs vinterhamn (Värmdö kn),740001312,18.704808,59.286511
1,Stavsnäs by (Värmdö kn),740024823,18.689473,59.28723
2,Styrsvik brygga (Värmdö kn),740025723,18.731488,59.280165
3,Styrsviks vsk (Ronneby kn),740032401,15.357657,56.137006
4,Nämdöböte brygga (Värmdö kn),740012544,18.73983,59.208557


In [61]:
import pickle

with open('dataset_30_station_ID.pickle', 'wb') as output:
    pickle.dump(df, output)

df.to_excel('dataset_30_station_ID.xlsx')

step 2 To get public transport (realtime info) for arrival and departure time at the searched ID

In [8]:
r2 = [0 for i in range(df.shape[0])]

for index, row in df.iterrows():
    urls2 = "https://api.resrobot.se/v2/departureBoard?key=YOURKEY&id=" + row['ID']
    r2[index] = requests.get(urls2)

print(r2[0].status_code)  


200


In [9]:
dfcols2 = ['name','stop','stopid','direction','transportNumber','date','time']
df2 = pd.DataFrame(columns = dfcols2)

for el in r2:
    root2 = ET.fromstring(el.content)

    for child in root2.iter('{hafas_rest_v1}Departure'):
        lineName = child.attrib['name']
        stopName = child.attrib['stop']
        stopID = child.attrib['stopid']
        lineDirection = child.attrib['direction']
        lineNumber = child.attrib['transportNumber']
        date = child.attrib['date']
        time = child.attrib['time']    
        df2 = df2.append(pd.Series([lineName,stopName,stopID,lineDirection,lineNumber,date,time], index = dfcols2), ignore_index = True)

df2.head()

#root2 = ET.fromstring(response2.content)
#for child in root2.iter('{hafas_rest_v1}Departure'):
#    print(child.tag,child.attrib)

,name,stop,stopid,direction,transportNumber,date,time
0,Länstrafik - Buss 434,Stavsnäs vinterhamn (Värmdö kn),740001312,Sollenkroka brygga (Värmdö kn),434,2020-05-03,13:34:00
1,Länstrafik - Buss 433,Stavsnäs vinterhamn (Värmdö kn),740001312,Slussen T-bana (Stockholm kn),433,2020-05-03,13:42:00
2,Länstrafik - Buss 434,Stavsnäs vinterhamn (Värmdö kn),740001312,Slussen T-bana (Stockholm kn),434,2020-05-03,13:42:00
3,Länstrafik - Färja 16,Stavsnäs vinterhamn (Värmdö kn),740001312,Hagede brygga (Värmdö kn),16,2020-05-03,14:10:00
4,Länstrafik - Buss 433,Stavsnäs vinterhamn (Värmdö kn),740001312,Slussen T-bana (Stockholm kn),433,2020-05-03,14:12:00


In [11]:
import pickle

with open('dataset_30_ID_departure_time.pickle', 'wb') as output:
    pickle.dump(df2, output)

df2.to_excel('dataset_30_ID_departure_time.xlsx')